In [10]:
import zipfile
import os

In [7]:
with zipfile.ZipFile('files_resized_choose.zip', 'r') as zip_ref:
    zip_ref.extractall('/workspace/Classifier/files_resized_choose')
    


In [6]:
os.path.abspath("")

'/workspace/Classifier'

In [9]:
with zipfile.ZipFile('files_choose.zip', 'r') as zip_ref:
    zip_ref.extractall('/workspace/Classifier/files_choose')

In [11]:
with zipfile.ZipFile('validation.zip', 'r') as zip_ref:
    zip_ref.extractall('/workspace/Classifier/CatDog')

In [12]:
with zipfile.ZipFile('CatPlusDog.zip', 'r') as zip_ref:
    zip_ref.extractall('/workspace/Classifier/CatPlusDog')

In [ ]:
### Calculate the mean and std of the dataset

In [70]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

import os
import pandas as pd
from torchvision.io import read_image
from torch.utils.data import Dataset, DataLoader

import numpy as np
import pandas as pd

from tqdm import tqdm

%matplotlib inline

In [20]:
# 12345->01234
# delete 4, delete 1, 23->1
label = np.load("Cas.npy")
label = label.reshape(4989,1)
label = label - 1

index = np.where(label==4)
label = np.delete(label, np.where(label == 4))

index_0 = np.where(label==0)
index_0 = index_0[0][:2800]
label = np.delete(label, index_0)

index_1 = np.where(label==1)
label = np.delete(label, index_1)

label[label==2] = 1
label[label==3] = 1

label = label.reshape(1491, 1)

In [21]:
for i in range(4):
    print(f"The number of images with label {i} is {np.where(label ==i)[0].shape}")

The number of images with label 0 is (780,)
The number of images with label 1 is (711,)
The number of images with label 2 is (0,)
The number of images with label 3 is (0,)


In [22]:
RESIZED_ROOT = os.path.join(os.path.dirname(os.path.abspath('')),'Classifier','files_choose')
path_list = os.listdir(RESIZED_ROOT)
path_list.sort(key = lambda x:int(x.split('.')[0]))

file = []
for file_name in path_list:
    file.append(file_name)

file = np.array(file)
file = np.delete(file, index)
file = np.delete(file, index_0)
file = np.delete(file, index_1)
file = file.reshape(1491,1)

annotation = np.concatenate((file,label),axis=1)


In [71]:
# df = pd.DataFrame(annotation)
# df = df.rename(columns={0: "image_id", 1: "label"})
# df

In [65]:
class CustomImageDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.DataFrame(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(img_path)
        image = image.to(torch.float)
        if self.transform:
            image = self.transform(image)
        return image

In [66]:
transform = transforms.Compose(
    [transforms.Resize((960,960),antialias=True),
    transforms.Normalize((0,0,0),(1,1,1))])
image_size = 960

In [67]:
data = CustomImageDataset(annotation,"files_choose",transform=transform)
dataloader = DataLoader(data, batch_size=8, shuffle=True, num_workers=4, pin_memory=True)

In [68]:
# placeholders
psum    = torch.tensor([0.0, 0.0, 0.0])
psum_sq = torch.tensor([0.0, 0.0, 0.0])

# loop through images
for inputs in tqdm(dataloader):
    psum    += inputs.sum(axis        = [0, 2, 3])
    psum_sq += (inputs ** 2).sum(axis = [0, 2, 3])

100%|██████████| 187/187 [00:16<00:00, 11.01it/s]


In [69]:
# pixel count
count = len(df) * image_size * image_size

# mean and std
total_mean = psum / count
total_var  = (psum_sq / count) - (total_mean ** 2)
total_std  = torch.sqrt(total_var)

# output
print('mean: '  + str(total_mean))
print('std:  '  + str(total_std))

mean: tensor([145.0785, 136.7515, 126.1401])
std:  tensor([52.9697, 52.6246, 55.6964])


In [59]:
psum_sq

tensor([1.4653e+11, 1.4566e+11, 1.4352e+11])

In [60]:
psum

tensor([1.9935e+11, 1.8791e+11, 1.7333e+11])

In [55]:
total_mean

tensor([145.0785, 136.7515, 126.1402])

In [58]:
psum_sq / 1374105600

tensor([106.6331, 106.0066, 104.4488])

In [57]:
count

1374105600